In [1]:
import tensorflow as tf
import numpy as np
import glob
from tqdm import tqdm
import random

FEATURES = [
    "u_component_of_wind_10m_above_ground",
    "v_component_of_wind_10m_above_ground",
    "temperature_2m_above_ground",
    "precipitable_water_entire_atmosphere",
]

TILE_SIZE = (128, 128)

#since too much data, getting stats for each record then the stats for those, less accurate but good enough

record_percentiles = {key: [] for key in FEATURES} 
record_means = {key: [] for key in FEATURES}
record_stds = {key: [] for key in FEATURES}

tfrecord_files = sorted(glob.glob("./data/data_full_train__*.tfrecord"))

2025-04-24 00:26:59.901979: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#iterate over files and get values
for tfrecord_path in tqdm(tfrecord_files):
    dataset = tf.data.TFRecordDataset(tfrecord_path)

    for raw_example in dataset:
        example = tf.train.Example()
        example.ParseFromString(raw_example.numpy())

        for key in FEATURES:
            try:
                values = example.features.feature[key].float_list.value
                if not values:
                    continue
                arr = np.array(values)
                record_percentiles[key].append([
                    np.percentile(arr, 0.1),
                    np.percentile(arr, 99.9)
                ])
                record_means[key].append(np.mean(arr))
                record_stds[key].append(np.std(arr))
            except KeyError:
                continue

100%|██████████| 34/34 [04:02<00:00,  7.12s/it]


In [3]:
record_percentiles

{'u_component_of_wind_10m_above_ground': [[-2.0586713373661043,
   -1.3824581491947174],
  [-1.7745551323890687, -0.8213583827018722],
  [0.23067215614020825, 4.135781378746034],
  [-2.030662353038788, -0.865148040652275],
  [0.0388854057714343, 4.185137267112733],
  [-5.205893189907074, -2.8730569350719453],
  [-3.343490136861801, -1.3684935152530604],
  [-2.036004800796509, 0.020990659338429245],
  [-1.8812464171648025, -0.9736325800418854],
  [-2.4853589379787446, -1.3531378602981563],
  [-2.654547802209854, -0.7019925943016977],
  [-3.1917628371715545, -2.292135057449341],
  [-2.069947074651718, -0.7688113334774966],
  [0.7784422299265862, 4.713726747035981],
  [-5.205893189907074, -2.8730569350719453],
  [0.23067215614020825, 4.140283472537997],
  [0.01428649520035833, 4.18954469203949],
  [-3.2559707403182983, -1.379982308149335],
  [-1.4089792531728744, -0.5958185753226279],
  [-5.205893189907074, -2.8730569350719453],
  [0.4083645525574684, 4.0860834193229705],
  [-1.9052482879

In [4]:
for key in FEATURES:
    p001s, p999s = zip(*record_percentiles[key])
    mean_p001 = np.mean(p001s)
    mean_p999 = np.mean(p999s)
    mean_mean = np.mean(record_means[key])
    mean_std = np.mean(record_stds[key])

    print(f"'{key}': ({mean_p001:.4f}, {mean_p999:.4f}, {mean_mean:.4f}, {mean_std:.4f}),")

'u_component_of_wind_10m_above_ground': (-0.1272, 1.4860, 0.6746, 0.3910),
'v_component_of_wind_10m_above_ground': (-0.6535, 0.9800, 0.1719, 0.3948),
'temperature_2m_above_ground': (15.2312, 18.6066, 16.9189, 0.8326),
'precipitable_water_entire_atmosphere': (16.0138, 18.7311, 17.3379, 0.6677),
